**RFM is essential for which customers is best or which customers need attention, discount or campaign. That is why I segmented all of my customers and make actions and also want reaction from my customers. With this project, you can easily define a segment for every type of customers and check you groups for actions.**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Let's start wiht importing libraries and preprocessing functions that we used before. I do not want to use na datas and that is why, in this project i would like to delete them.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

df_ = pd.read_csv("../input/online-retail-ii-uci/online_retail_II.csv")

df = df_.copy()


df.info()




df.isnull().sum()

df.dropna(inplace=True)

df.nunique()

df.nunique().sum()

df["Quantity"].groupby(df["Description"]).agg("sum")


df["Quantity"].groupby(df["Description"]).agg("sum").sort_values(ascending=False).head()

df.head()

df[df["Invoice"].str.contains("C", na=False)].head()






**Cancelled operations is defined with minus quantity and minus price. That is why, I would like to drop with contains function.**

In [ ]:
df = df[~df["Invoice"].str.contains("C", na=False)]

df.head()
df["TotalPrice"] = df["Quantity"] * df["Price"]


df["InvoiceDate"].max()

**Now, let's count today :) We decided the today_date is close to dataset's last date. You can arrange it whatever you want but this way is more healthy. For RFM segmentation, aggregation functions is crucial and we can see that recency and frequency 
is really important for us. Monetary just decide a small part of RFM.**

In [ ]:
today_date =dt.datetime(2011, 12, 11)

df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

rfm = df.groupby("Customer ID").agg({"InvoiceDate":lambda InvoiceDate: (today_date - InvoiceDate.max()).days,
                                     "Invoice": lambda Invoice : Invoice.nunique(),
                                     "TotalPrice": lambda TotalPrice : TotalPrice.sum()})

rfm.columns = ['recency', 'frequency', 'monetary']

rfm = rfm[rfm["monetary"] > 0]



rfm["recency_sc"] = pd.qcut(rfm["recency"],5,labels=[5,4,3,2,1])
rfm["frequency_sc"] = pd.qcut(rfm["frequency"].rank(method="first"),5,labels=[1,2,3,4,5])
rfm["monetary_sc"] = pd.qcut(rfm["monetary"],5,labels=[1,2,3,4,5])

rfm["RFM_SCORE"] = (rfm["recency_sc"].astype(str) + rfm["frequency_sc"].astype(str))

rfm[rfm["RFM_SCORE"]=="11"].head()

In [ ]:
rfm[rfm["RFM_SCORE"] == "55"].head()


**See, we arranged best customers with "55" and hibernating customers defined with "11". In my opinion,  "33" segment is essential and you can make actions for them. If you can make them loyal customer, Mission Completed :) Thank you for your time.**